In [1]:
# Processing tools to read from Helsinki WFS API and produce GeoJSON

import json

import requests


# Helsinki district division data

Helsinki major district, district and city sub-district data is available
from Helsinki Region Infoshare (HRI) as WFS API in GeoJSON format.
The data is available in Finnish and Swedish and in EPSG:4326 (WGS84) projection.

https://hri.fi/data/en_GB/dataset/helsingin-piirijako


In [2]:
# dataset name mapping to file name (major district, district, city sub-district)
district_mapping = {
"Piirijako_suurpiiri": "HelsinkiMajorDistrict",
"Piirijako_peruspiiri": "HelsinkiDistrict",
"Piirijako_osaalue": "HelsinkiSubDistrict",
}

for type_name in district_mapping.keys():
    res = requests.get(
        f"https://kartta.hel.fi/ws/geoserver/avoindata/wfs?service=wfs&version=2.0.0&request=GetFeature&typeNames=avoindata:{type_name}&count=1000000&outputFormat=application/json&srsName=urn:ogc:def:crs:EPSG::4326"
    )
    # parse json to make sure it is valid json
    data = res.json()
    with open(f"{district_mapping[type_name]}.json", "w") as f:
        json.dump(data, f)
